In [1]:
# Regular expression to select files
file_regex = "*Sample*.zip"

In [2]:
# Import libraries
from pathlib import Path
from tempfile import TemporaryDirectory
from typing import List
import pandas as pd
import imcsegpipe
from imcsegpipe.utils import sort_channels_by_mass

# Working directory storing all outputs
work_dir = Path("analysis")
work_dir.mkdir(exist_ok=True)

# Set and create output directories
acquisitions_dir = work_dir / "extracted_mcd"
segment_dir = work_dir / "for_segmentation"
output_dir = work_dir / "full_images"
acquisitions_dir.mkdir(exist_ok=True)
segment_dir.mkdir(exist_ok=True)
output_dir.mkdir(exist_ok=True)

# Extract .mcd files to 'extracted_mcd' folder
temp_dirs = []
try:
    for raw_dir in [Path("raw")]:
        zip_files = list(raw_dir.rglob(file_regex))
        if len(zip_files) > 0:
            temp_dir = TemporaryDirectory()
            temp_dirs.append(temp_dir)
            for zip_file in sorted(zip_files):
                imcsegpipe.extract_zip_file(zip_file, temp_dir.name)
    acquisition_metadatas = []
    for raw_dir in [Path("raw")] + [Path(temp_dir.name) for temp_dir in temp_dirs]:
        mcd_files = list(raw_dir.rglob("*.mcd"))
        mcd_files = [(i) for i in mcd_files if not i.stem.startswith('.')]
        if len(mcd_files) > 0:
            txt_files = list(raw_dir.rglob("*.txt"))
            txt_files = [(i) for i in txt_files if not i.stem.startswith('.')]
            matched_txt_files = imcsegpipe.match_txt_files(mcd_files, txt_files)
            for mcd_file in mcd_files:
                imcsegpipe.extract_mcd_file(
                    mcd_file,
                    acquisitions_dir / mcd_file.stem,
                    txt_files=matched_txt_files[mcd_file]
                )
finally:
    for temp_dir in temp_dirs:
        temp_dir.cleanup()
    del temp_dirs

# Generate image stacks containing all channels (_full) and only channels for segmentation (_segment)
panel = pd.read_csv("raw/panel.csv")
for acquisition_dir in acquisitions_dir.glob("[!.]*"):
    if acquisition_dir.is_dir():
        imcsegpipe.create_analysis_stacks(
            acquisition_dir=acquisition_dir,
            analysis_dir=output_dir,
            analysis_channels=sort_channels_by_mass(
                panel.loc[panel["Full"] == 1, "Metal Tag"].tolist()
            ),
            suffix="_full",
            hpf=50.0
        )
        imcsegpipe.create_analysis_stacks(
            acquisition_dir=acquisition_dir,
            analysis_dir=segment_dir,
            analysis_channels=sort_channels_by_mass(
                panel.loc[panel["Segment"] == 1, "Metal Tag"].tolist()
            ),
            suffix="_segment",
            hpf=50.0
        )

c:\Users\oscar\anaconda3\envs\imcsegpipe\lib\site-packages\readimc\mcd_parser.py:131: UserWarning: Slide 0 corrupted: overlapping memory blocks for acquisitions 1 and 2
  warn(
